# Import Libraries

In [3]:
import os
import sys
import time
import math
import pickle
import random
import pathlib
import sqlite3
import tempfile
import importlib
import subprocess
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt
from configparser import ConfigParser
from distutils.spawn import find_executable
from collections import OrderedDict, namedtuple
from mlxtend.feature_selection import ColumnSelector

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import torch.nn.functional as F
from torch.autograd import Variable
from sru import SRU, SRUCell

# Set Parameters

In [4]:
np.random.seed(0)

torch.cuda.empty_cache()
torch.manual_seed(0)
torch.backends.cudnn.benchmark=True

os.environ['CUDA_VISIBLE_DEVICES'] = '2' # GPU Number
os.environ['TORCH_EXTENSIONS_DIR'] = os.popen('pwd').read().strip('\n') # SRU Temp Folder

In [5]:
# Mean and Standard Deviation for Noise - used for SNR analysis
scr4_mean = 31.827541
scr4_std = 7.5468507

In [6]:
NUM_EPOCHS = 10 # No Of Training Epochs
TRAIN_BATCH_SIZE = 1024 # Batch size for Training
VAL_BATCH_SIZE = 128 # Batch size for Validation and Testing

# Parameters for RNN based Architectures
EMBEDDING_DIM = 2
HIDDEN_DIM = 100
TAGSET_SIZE = 2

# DNN Modules

In [19]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(2, 50)
        self.selu = nn.SELU()
        self.linear2 = nn.Linear(50, 50)
        self.selu = nn.SELU()
        self.linear3 = nn.Linear(50, 50)
        self.selu = nn.SELU()        
        self.drop1 = nn.AlphaDropout(p=0.5)
        self.linear5 = nn.Linear(50,2)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input):
        output = self.linear1(input)
        output = self.selu(output)
        output = self.drop1(output)
        output = self.linear2(output)
        output = self.selu(output)
        output = self.drop1(output)
        output = self.linear3(output)
        output = self.selu(output)
        output = self.drop1(output)
        output = self.linear5(output)
        output = self.softmax(output)
        return output

In [9]:
class GRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(GRUTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.gru = nn.GRU(embedding_dim, hidden_dim)        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
            
    def forward(self, sentence):
        gru_out, _ = self.gru(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        return tag_space
    

In [10]:
class BGRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(BGRUTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.bgru = nn.GRU(embedding_dim, hidden_dim,bidirectional=True)        
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
            
    def forward(self, sentence):
        gru_out, _ = self.bgru(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        return tag_space
    

In [11]:
class BSRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(BSRUTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.bsru = SRU(embedding_dim, hidden_dim,2,bidirectional=True)        
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
        
    
    def forward(self, sentence):
        sru_out, _ = self.bsru(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(sru_out.view(len(sentence), -1))
        return tag_space
    

In [12]:
class CBSDNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        nn.init.xavier_normal_(self.conv1.weight)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        nn.init.xavier_normal_(self.conv2.weight)

        self.linear1 = nn.Linear(16*2, 100)
        
        self.bsru = SRU(input_size=100, hidden_size=100,num_layers=2,bidirectional=True)
        self.linear2 = nn.Linear(100*2, 100*2)
        nn.init.xavier_normal_(self.linear2.weight)

        self.linear3 = nn.Linear(100*2, 2)

        self.relu = nn.ReLU()

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, input):
        
        output = self.conv1(input)
        output = self.relu(output)

        output = self.conv2(output)
        output = self.relu(output)
        output = self.dropout1(output)
                
        output = output.view(-1, 16*2)
        output1 = self.linear1(output)

        output = output1.view(len(output1),1 ,-1 )
        output, _ = self.bsru(output)
        
        output = output.view(len(output1), -1)

        output = self.linear2(output)
        output = self.relu(output)

        output = self.dropout2(output)

        output = self.linear3(output)
        return output


# Acquire and Prepare Data

In [10]:
with open('scrimmage4_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [14]:
# Retain only the features selected by the RFE method
# Columns : 0 - SNR, 1 - MCS, 2 - Center Frequency, 3 - Bandwidth, [4:19] - PSD
cols = torch.LongTensor([0,1]) 
link_data = [(link_datas[0][:,cols], link_datas[1]) for link_datas in link_dataset] 

In [15]:
# Without Pilot
test_start = 142
total_links = len(link_dataset)
# Do a 40:10:50 train:validation:test split 
train_data = link_data[:114]
val_data = link_data[114:142]
test_data = link_data[142:]

train_x = torch.cat(tuple(link[0] for link in train_data),dim=0)
train_y = torch.cat(tuple(link[1] for link in train_data),dim=0)

val_x = torch.cat(tuple(link[0] for link in val_data),dim=0)
val_y = torch.cat(tuple(link[1] for link in val_data),dim=0)

test_x = torch.cat(tuple(link[0] for link in test_data),dim=0)
test_y = torch.cat(tuple(link[1] for link in test_data),dim=0)

# Reshape data for CBSDNN
train_x_cbsdnn = train_x.view(-1,1,2)
val_x_cbsdnn = val_x.view(-1,1,2)
test_x_cbsdnn = test_x.view(-1,1,2)

In [16]:
# With Pilot

datadist = [0.4, 0.1, 0.5] # distribution among train, test and validation

train_x_npn = []
train_y_npn = []
val_x_npn = []
val_y_npn = []
test_x_npn = []
test_y_npn = []

for i in range(len(link_data)):
    datalen = len(link_data[i][1])
    trainlen = int(datalen*datadist[0])
    vallen = int(datalen*(datadist[0]+datadist[1]))

    train_x_npn.append(link_data[i][0][0:trainlen].numpy())    
    train_y_npn.append(link_data[i][1][0:trainlen].numpy())
    
    val_x_npn.append(link_data[i][0][trainlen:vallen].numpy())    
    val_y_npn.append(link_data[i][1][trainlen:vallen].numpy())
    
    test_x_npn.append(link_data[i][0][vallen:datalen].numpy())
    test_y_npn.append(link_data[i][1][vallen:datalen].numpy())

train_x_np = np.concatenate(train_x_npn)
train_y_np = np.concatenate(train_y_npn)
val_x_np = np.concatenate(val_x_npn)
val_y_np = np.concatenate(val_y_npn)
test_x_np = np.concatenate(test_x_npn)
test_y_np = np.concatenate(test_y_npn)

train_x_plt = torch.from_numpy(train_x_np)
val_x_plt = torch.from_numpy(val_x_np)
test_x_plt = torch.from_numpy(test_x_np)
train_y_plt = torch.from_numpy(train_y_np)
val_y_plt = torch.from_numpy(val_y_np)
test_y_plt = torch.from_numpy(test_y_np)

# Reshape data for CBSDNN
train_x_plt_cbsdnn = train_x_plt.view(-1,1,2)
val_x_plt_cbsdnn = val_x_plt.view(-1,1,2)
test_x_plt_ccbsnn = test_x_plt.view(-1,1,2)

# Running Tests

In [17]:
# Initialize Result Variables
mlp_scr4_res = {}
gru_scr4_res = {}
bgru_scr4_res = {}
bsru_scr4_res = {}
cbsdnn_scr4_res = {}

# Without Pilot

# MLP

In [20]:
# Train

model = MLP().cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=True, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

model.train()
for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
   
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.63080, validation loss: 0.58914


Epoch 3/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.55983, validation loss: 0.54588


Epoch 4/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

...

train loss: 0.55072, validation loss: 0.54343


Epoch 10/10, Validation:   0%|                                 | 0/4757 [00:00<?, ?it/s]

train loss: 0.55040, validation loss: 0.54366


Epoch 10/10, Validation: 100%|█████████████████████| 4757/4757 [00:15<00:00, 315.37it/s]

train loss: 0.55010, validation loss: 0.54104


In [23]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = [] # Predicted Label
target = [] # Target Lable
snrss = [] # SNR (Calculated from Noise Variance)
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

mlp_scr4_res['mlp_scr4_prd'] = predict
mlp_scr4_res['mlp_scr4_trg'] = target
mlp_scr4_res['mlp_scr4_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 26457/26457 [00:33<00:00, 796.49it/s]


In [ ]:
# Test Accuracy per Link

accr = []
for i in range(total_links - test_start):
    test_data_i = [link_data[i+test_start]]
    test_x_i = torch.cat(tuple(link[0] for link in test_data_i),dim=0)
    test_y_i = torch.cat(tuple(link[1] for link in test_data_i),dim=0)
    test_x_i = test_x_i.view(-1,1,2)
    
    test_dataloader = data.DataLoader(
    data.TensorDataset(test_x_i,test_y_i), 
    batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

    progress_test = tqdm(
        test_dataloader, 
        desc=f'Link {i+test_start+1}/{total_links}, Test',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)

    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_test):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    tp = predict[target==1].sum()
    tn = (target==0).sum() - predict[target==0].sum()
    fp = predict[target==0].sum()
    fn = (target==1).sum() - predict[target==1].sum()
    accr.append((tp+tn)/(tp+tn+fp+fn))    

mlp_scr4_res['mlp_scr4_accr_perlink'] = accr

# GRU

In [18]:
# Train

model = GRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',w
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.54342, validation loss: 0.52424


Epoch 3/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.52892, validation loss: 0.51896


Epoch 4/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

...

train loss: 0.51658, validation loss: 0.51290


Epoch 10/10, Validation:   0%|                                 | 0/4757 [00:00<?, ?it/s]

train loss: 0.51418, validation loss: 0.50905


Epoch 10/10, Validation: 100%|██████████████████████| 4757/4757 [01:10<00:00, 67.19it/s]

train loss: 0.51165, validation loss: 0.50444


In [19]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

gru_scr4_res['gru_scr4_prd'] = predict
gru_scr4_res['gru_scr4_trg'] = target
gru_scr4_res['gru_scr4_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 26457/26457 [00:56<00:00, 471.91it/s]


In [20]:
# Test Accuracy per Link

accr = []
for i in range(total_links - test_start):
    test_data_i = [link_data[i+test_start]]
    test_x_i = torch.cat(tuple(link[0] for link in test_data_i),dim=0)
    test_y_i = torch.cat(tuple(link[1] for link in test_data_i),dim=0)
    
    test_dataloader = data.DataLoader(
    data.TensorDataset(test_x_i,test_y_i), 
    batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)
    
    progress_test_epoch = tqdm(
        test_data_i, 
        desc=f'Link {i+test_start+1}/{total_links}, Test',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_data_i), smoothing=.9)

    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_test_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())

    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    tp = predict[target==1].sum()
    tn = (target==0).sum() - predict[target==0].sum()
    fp = predict[target==0].sum()
    fn = (target==1).sum() - predict[target==1].sum()
    accr.append((tp+tn)/(tp+tn+fp+fn))    

gru_scr4_res['gru_scr4_accr_perlink'] = accr

Link 284/284, Test: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  3.03it/s]


# BGRU

In [21]:
# Train

model = BGRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.52000, validation loss: 0.47813


Epoch 3/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.45005, validation loss: 0.41727


Epoch 4/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

...

train loss: 0.41534, validation loss: 0.39123


Epoch 10/10, Validation:   0%|                                 | 0/4757 [00:00<?, ?it/s]

train loss: 0.41430, validation loss: 0.39090


Epoch 10/10, Validation: 100%|██████████████████████| 4757/4757 [02:09<00:00, 36.64it/s]

train loss: 0.41339, validation loss: 0.39069


In [22]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bgru_scr4_res['bgru_scr4_prd'] = predict
bgru_scr4_res['bgru_scr4_trg'] = target
bgru_scr4_res['bgru_scr4_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 26457/26457 [01:26<00:00, 305.08it/s]


In [23]:
# Test Accuracy per Link

accr = []

for i in range(total_links - test_start):
    test_data_i = [link_data[i+test_start]]
    test_x_i = torch.cat(tuple(link[0] for link in test_data_i),dim=0)
    test_y_i = torch.cat(tuple(link[1] for link in test_data_i),dim=0)
    
    test_dataloader = data.DataLoader(
    data.TensorDataset(test_x_i,test_y_i), 
    batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)
    
    progress_test_epoch = tqdm(
        test_data_i, 
        desc=f'Link {i+test_start+1}/{total_links}, Test',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_data_i), smoothing=.9)

    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_test_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())

    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    tp = predict[target==1].sum()
    tn = (target==0).sum() - predict[target==0].sum()
    fp = predict[target==0].sum()
    fn = (target==1).sum() - predict[target==1].sum()
    accr.append((tp+tn)/(tp+tn+fp+fn))    

bgru_scr4_res['bgru_scr4_accr_perlink'] = accr

Link 284/284, Test: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  1.56it/s]


# BSRU

In [24]:
# Train

model = BSRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.50173, validation loss: 0.46138


Epoch 3/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.45479, validation loss: 0.44432


Epoch 4/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

...

train loss: 0.41537, validation loss: 0.41120


Epoch 10/10, Validation:   0%|                                 | 0/4757 [00:00<?, ?it/s]

train loss: 0.41357, validation loss: 0.40897


Epoch 10/10, Validation: 100%|█████████████████████| 4757/4757 [00:21<00:00, 222.18it/s]

train loss: 0.41199, validation loss: 0.40753


In [25]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bsru_scr4_res['bsru_scr4_prd'] = predict
bsru_scr4_res['bsru_scr4_trg'] = target
bsru_scr4_res['bsru_scr4_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 26457/26457 [00:30<00:00, 869.85it/s]


In [26]:
# Test Accuracy per Link

accr = []

for i in range(total_links - test_start):
    test_data_i = [link_data[i+test_start]]
    test_x_i = torch.cat(tuple(link[0] for link in test_data_i),dim=0)
    test_y_i = torch.cat(tuple(link[1] for link in test_data_i),dim=0)
    
    test_dataloader = data.DataLoader(
    data.TensorDataset(test_x_i,test_y_i), 
    batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)
    
    progress_test_epoch = tqdm(
        test_data_i, 
        desc=f'Link {i+test_start+1}/{total_links}, Test',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_data_i), smoothing=.9)

    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_test_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())

    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    tp = predict[target==1].sum()
    tn = (target==0).sum() - predict[target==0].sum()
    fp = predict[target==0].sum()
    fn = (target==1).sum() - predict[target==1].sum()
    accr.append((tp+tn)/(tp+tn+fp+fn))    

bsru_scr4_res['bsru_scr4_accr_perlink'] = accr

Link 284/284, Test: 100%|█████████████████████████████████| 1/1 [00:00<00:00, 17.50it/s]


# CBSDNN

In [27]:
# Train
model = CBSDNN().cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_cbsdnn,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_cbsdnn,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_cbsdnn,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

start_time = time.time()

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()

    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
  
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 1/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.51335, validation loss: 0.47881


Epoch 2/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.45970, validation loss: 0.44166


Epoch 3/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

...

train loss: 0.41666, validation loss: 0.40545


Epoch 9/10, Validation:   0%|                                  | 0/4757 [00:00<?, ?it/s]

train loss: 0.41396, validation loss: 0.40365


Epoch 9/10, Validation: 100%|██████████████████████| 4757/4757 [00:27<00:00, 172.38it/s]

train loss: 0.41118, validation loss: 0.40090


In [28]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0][:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

cbsdnn_scr4_res['cbsdnn_scr4_prd'] = predict
cbsdnn_scr4_res['cbsdnn_scr4_trg'] = target
cbsdnn_scr4_res['cbsdnn_scr4_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 26457/26457 [00:38<00:00, 682.02it/s]


In [29]:
# Test Accuracy per Link

accr = []


for i in range(total_links - test_start):
    test_data_i = [link_data[i+test_start]]
    test_x_i = torch.cat(tuple(link[0] for link in test_data_i),dim=0)
    test_y_i = torch.cat(tuple(link[1] for link in test_data_i),dim=0)
    test_x_i = test_x_i.view(-1,1,2)
    
    test_dataloader = data.DataLoader(
    data.TensorDataset(test_x_i,test_y_i), 
    batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=16, pin_memory=True)

    progress_test = tqdm(
        test_dataloader, 
        desc=f'Link {i+test_start+1}/{total_links}, Test',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(test_dataloader), smoothing=.9)

    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_test):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    tp = predict[target==1].sum()
    tn = (target==0).sum() - predict[target==0].sum()
    fp = predict[target==0].sum()
    fn = (target==1).sum() - predict[target==1].sum()
    accr.append((tp+tn)/(tp+tn+fp+fn))    

cbsdnn_scr4_res['cbsdnn_scr4_accr_perlink'] = accr

Link 284/284, Test: 100%|████████████████████████████| 247/247 [00:01<00:00, 192.95it/s]


# With Pilot

# MLP

In [35]:
# Train

model = MLP().cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=True, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

model.train()

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
   
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.54985, validation loss: 0.66600


Epoch 3/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.53042, validation loss: 0.60135


Epoch 4/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

...

train loss: 0.52855, validation loss: 0.59623


Epoch 10/10, Validation:   0%|                                 | 0/5096 [00:00<?, ?it/s]

train loss: 0.52848, validation loss: 0.59764


Epoch 10/10, Validation: 100%|█████████████████████| 5096/5096 [00:11<00:00, 445.37it/s]

train loss: 0.52842, validation loss: 0.59580


In [36]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = [] # Predicted Label
target = [] # Target Lable
snrss = [] # SNR (Calculated from Noise Variance)
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

mlp_scr4_res['mlp_scr4_plt_prd'] = predict
mlp_scr4_res['mlp_scr4_plt_trg'] = target
mlp_scr4_res['mlp_scr4_plt_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|████████████████████████| 25479/25479 [00:20<00:00, 1243.13it/s]


# GRU

In [37]:
# Train

model = GRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.52005, validation loss: 0.60013


Epoch 3/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.50405, validation loss: 0.58992


Epoch 4/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

...

train loss: 0.48659, validation loss: 0.56152


Epoch 10/10, Validation:   0%|                                 | 0/5096 [00:00<?, ?it/s]

train loss: 0.48498, validation loss: 0.55953


Epoch 10/10, Validation: 100%|██████████████████████| 5096/5096 [01:16<00:00, 66.53it/s]

train loss: 0.48379, validation loss: 0.55768


In [38]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

gru_scr4_res['gru_scr4_plt_prd'] = predict
gru_scr4_res['gru_scr4_plt_trg'] = target
gru_scr4_res['gru_scr4_plt_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 25479/25479 [00:56<00:00, 454.36it/s]


# BGRU

In [39]:
# Train

model = BGRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.48156, validation loss: 0.54465


Epoch 3/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.40024, validation loss: 0.52120


Epoch 4/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

...

train loss: 0.37810, validation loss: 0.47863


Epoch 10/10, Validation:   0%|                                 | 0/5096 [00:00<?, ?it/s]

train loss: 0.37692, validation loss: 0.47706


Epoch 10/10, Validation: 100%|██████████████████████| 5096/5096 [02:13<00:00, 38.04it/s]


train loss: 0.37584, validation loss: 0.47586


In [40]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bgru_scr4_res['bgru_scr4_plt_prd'] = predict
bgru_scr4_res['bgru_scr4_plt_trg'] = target
bgru_scr4_res['bgru_scr4_plt_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 25479/25479 [01:27<00:00, 292.38it/s]


# BSRU

In [41]:
# Train

model = BSRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.47413, validation loss: 0.53364


Epoch 3/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.41861, validation loss: 0.50588


Epoch 4/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

...

train loss: 0.38453, validation loss: 0.47428


Epoch 10/10, Validation:   0%|                                 | 0/5096 [00:00<?, ?it/s]

train loss: 0.38282, validation loss: 0.47242


Epoch 10/10, Validation: 100%|█████████████████████| 5096/5096 [00:22<00:00, 222.59it/s]

train loss: 0.38138, validation loss: 0.47104


In [42]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bsru_scr4_res['bsru_scr4_plt_prd'] = predict
bsru_scr4_res['bsru_scr4_plt_trg'] = target
bsru_scr4_res['bsru_scr4_plt_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 25479/25479 [00:30<00:00, 827.27it/s]


# CBSDNN

In [43]:
# Train
model = CBSDNN().cuda()
loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt_cbsdnn,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt_cbsdnn,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt_cbsdnn,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

start_time = time.time()

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()

    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
  
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 1/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.47954, validation loss: 0.59755


Epoch 2/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.41107, validation loss: 0.54286


Epoch 3/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

...

train loss: 0.38365, validation loss: 0.48927


Epoch 9/10, Validation:   0%|                                  | 0/5096 [00:00<?, ?it/s]

train loss: 0.38216, validation loss: 0.48565


Epoch 9/10, Validation: 100%|██████████████████████| 5096/5096 [00:28<00:00, 179.84it/s]


train loss: 0.38102, validation loss: 0.48474


In [45]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0][:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

cbsdnn_scr4_res['cbsdnn_scr4_plt_prd'] = predict
cbsdnn_scr4_res['cbsdnn_scr4_plt_trg'] = target
cbsdnn_scr4_res['cbsdnn_scr4_plt_snr'] = snrss * scr4_std + scr4_mean

Epoch 10/10, Test: 100%|█████████████████████████| 25479/25479 [00:38<00:00, 653.99it/s]


# Save Output Files

In [46]:
# MLP
outfile = os.path.join(os.getcwd(), 'mlp_scr4_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(mlp_scr4_res, file)
    
# GRU
outfile = os.path.join(os.getcwd(), 'gru_scr4_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(gru_scr4_res, file)
    
# BGRU
outfile = os.path.join(os.getcwd(), 'bgru_scr4_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(bgru_scr4_res, file)
    
# BSRU
outfile = os.path.join(os.getcwd(), 'bsru_scr4_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(bsru_scr4_res, file)

# CBSDNN
outfile = os.path.join(os.getcwd(), 'cbsdnn_scr4_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(cbsdnn_scr4_res, file)